In [2]:
import os
import pandas as pd
from sqlalchemy import create_engine, text
from dotenv import load_dotenv

load_dotenv()                          # pulls .env into the environment

PG_user = os.getenv("PG_USER")
PG_password = os.getenv("PG_PASSWORD")
PG_host = os.getenv("PG_HOST")
PG_DB = os.getenv("PG_DB")

engine = create_engine(
    f"postgresql://{PG_user}:{PG_password}@{PG_host}/{PG_DB}"
)

# (optional) default schema if you like:
with engine.begin() as conn:
    conn.execute(text("SET search_path TO sql_project;"))

pd.set_option("display.max_rows", None)


## Query 1 – Descriptive Analytics  
**Business Question:** *Which models receive the most complaints, and how concentrated is the volume?*

In [3]:
sql_query = '''
-- Total complaints and rank per make–model
WITH agg AS (
  SELECT
    "Make",
    "Model",
    SUM("Complaint count") AS total_complaints
  FROM "car_complaints"
  GROUP BY "Make", "Model"
)
SELECT
  "Make",
  "Model",
  total_complaints,
  DENSE_RANK() OVER (ORDER BY total_complaints DESC) AS complaint_rank
FROM agg
ORDER BY total_complaints DESC;
'''
df1 = pd.read_sql(sql_query, engine)
df1


,Make,Model,total_complaints,complaint_rank
0,Toyota,Camry,7194.0,1
1,Toyota,RAV4,5282.0,2
2,Toyota,Corolla,3110.0,3
3,Toyota,Prius,3052.0,4
4,Toyota,Tacoma,2302.0,5
5,Toyota,Sienna,1978.0,6
6,Toyota,Highlander,1602.0,7
7,Toyota,4Runner,1294.0,8
8,Toyota,Avalon,1172.0,9
9,Toyota,Tundra,1060.0,10


### Insight  
Just five Toyota nameplates (Camry 7 194 complaints, RAV4 5 282, Corolla 3 110, Prius 3 052, Tacoma 2 302) account for 64% of all Toyota complaints. The Camry alone represents most of the total.

### Recommendation  
Direct quality-improvement resources first to those five models—especially the Camry—before dispersing efforts across lower-volume vehicles.

### Prediction  
If corrective actions reduce complaints for each of the top-five models by even 20%, Toyota’s overall complaint count should fall by roughly 13% in the next reporting cycle.

## Query 2 – Diagnostic Analytics  
**Business Question:** For Toyota, what share of the make’s total complaints does each model contribute?  

In [4]:
sql_query = '''
WITH make_total AS (
  SELECT
    "Make",
    SUM("Complaint count") AS make_complaints
  FROM "car_complaints"
  WHERE "Make" = 'Toyota'
  GROUP BY "Make"
),
model_share AS (
  SELECT
    c."Model",
    SUM(c."Complaint count") AS model_complaints,
    m.make_complaints
  FROM "car_complaints" c
  JOIN make_total m
         ON m."Make" = c."Make"
  WHERE c."Make" = 'Toyota'
  GROUP BY c."Model", m.make_complaints
)
SELECT
  "Model",
  model_complaints,
  ROUND(100.0 * model_complaints / make_complaints, 2) AS pct_of_toyota_complaints
FROM model_share
ORDER BY model_complaints DESC;
'''
df2 = pd.read_sql(sql_query, engine)
df2


,Model,model_complaints,pct_of_toyota_complaints
0,Camry,7194.0,21.91
1,RAV4,5282.0,16.09
2,Corolla,3110.0,9.47
3,Prius,3052.0,9.30
4,Tacoma,2302.0,7.01
5,Sienna,1978.0,6.02
6,Highlander,1602.0,4.88
7,4Runner,1294.0,3.94
8,Avalon,1172.0,3.57
9,Tundra,1060.0,3.23


### Insight  
Toyota’s complaint volume is heavily concentrated: Camry (21.9 %) and RAV4 (16.1 %) together account for nearly 38 % of all Toyota complaints, while the remaining 80+ models each contribute < 10 %.

### Recommendation  
Prioritise corrective actions and customer-care initiatives on the Camry and RAV4 first; a proportional reduction on these two models will deliver the largest overall impact for the least effort.

### Prediction  
If Camry and RAV4 complaints are cut by 15 % each, Toyota’s total complaint count should fall by roughly 6% (0.38 × 0.15) in the next reporting cycle—even if every other model stays flat.
